In [10]:
from utils import *
import datasets
import os
import keras

/home/jose/scratch/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#!./download_cornell.sh

--2018-03-31 15:47:27--  https://github.com/Conchylicultor/DeepQA/raw/master/data/cornell/movie_conversations.txt
Resolviendo github.com (github.com)... 192.30.253.113, 192.30.253.112
Conectando con github.com (github.com)[192.30.253.113]:443... conectado.
Petición HTTP enviada, esperando respuesta... 302 Found
Ubicación: https://raw.githubusercontent.com/Conchylicultor/DeepQA/master/data/cornell/movie_conversations.txt [siguiente]
--2018-03-31 15:47:27--  https://raw.githubusercontent.com/Conchylicultor/DeepQA/master/data/cornell/movie_conversations.txt
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[151.101.16.133]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 6760930 (6.4M) [text/plain]
Guardando como: “movie_conversations.txt”

movie_conversations 100%[===================>]   6.45M   317KB/s    en 11s     

2018-03-31 15:47:39 (627 KB/s) - “mov

In [7]:
data = datasets.readCornellData("data/cornell", max_len=15)

100%|██████████| 83097/83097 [00:03<00:00, 23481.09it/s]


In [39]:
# Tokenization
from collections import Counter
from itertools import chain
start_symbol = '^'
end_symbol = '$'
padding_symbol = '#'
unk_symbol="|"
all_words = [start_symbol, end_symbol, padding_symbol,unk_symbol]+list(Counter(
    chain.from_iterable(map(lambda x: x.split(), chain.from_iterable(data)))).keys())

In [40]:
word2id = {symbol:i for i, symbol in enumerate(all_words)}
id2word = {i:symbol for symbol, i in word2id.items()}

In [104]:
def w2i(sentence, word2id=word2id, padded_len=10):
    """ Converts a sequence of words to a padded sequence of their ids.

      sentence: a string, input/output sequence of symbols.
      word2id: a dict, a mapping from original symbols to ids.
      padded_len: an integer, a desirable length of the sequence.

      result: a tuple of (a list of ids, an actual length of sentence).
    """
    # Account for start and end
    
    sentence = sentence.split()
    padded_len = padded_len+2
    sentence = ["^"]+sentence
    sent_ids = [word2id[i] if i in word2id.keys() else word2id["|"]
                for i in sentence]
    sent_len = len(sent_ids[:padded_len-1])+1
    sent_ids = sent_ids[:padded_len-1]+[word2id["$"]] + \
        [word2id["#"]]*(padded_len-len(sent_ids)-1)

   # return (sent_ids, sent_len-2)
    return sent_ids

In [157]:
def w2i_2(sentence, word2id=word2id, padded_len=10):
    sentence = sentence.split()
    padded_len = padded_len+2
    sent_ids = [word2id[i] if i in word2id.keys() else word2id["|"]
                for i in sentence]
    sent_len = len(sent_ids[:padded_len-1])+1
    sent_ids = sent_ids[:padded_len-1]+ \
        [word2id["#"]]*(padded_len-len(sent_ids)-1)

   # return (sent_ids, sent_len-2)
    return sent_ids

In [53]:
def ids_to_sentence(ids, id2word):
    """ Converts a sequence of ids to a sequence of symbols.
    
          ids: a list, indices for the padded sequence.
          id2word:  a dict, a mapping from ids to original symbols.

          result: a list of symbols.
    """
 
    return [id2word[i] for i in ids] 

In [203]:
y_ohot=[i[1] for i in data]
y_ohot

['where',
 'the real you',
 'lets go',
 'i hope so',
 'they do not',
 'joey',
 'expensive',
 'you set me up',
 'i just wanted',
 'so they tell me',
 'get out',
 'then go get her',
 'where',
 'not at all',
 'away',
 'a deals a deal',
 'do what',
 'are you lost',
 'who cares',
 'im fine im',
 'why',
 'dorsey',
 'i hate him',
 'what',
 'no you werent',
 'this',
 'bianca',
 'pleasant',
 'unwelcome',
 'for',
 'fallacy',
 'the duck',
 'hearsay',
 'no what',
 'what',
 'which parts',
 'oh yeah',
 'right',
 'kat a fan too',
 'yeah',
 'what',
 'good enough',
 'thank god',
 'no',
 'why',
 'so was christ',
 'but its',
 'divorced',
 'takin a bath',
 'any id',
 'its okay',
 'no im okay',
 'hmmmm',
 'my folks are',
 'stay here',
 'better than one',
 'im on duty',
 'so am i',
 'alright',
 'alright',
 'alright',
 'okay',
 'hey',
 'yeah',
 'you promise',
 'yeah i promise',
 'dont be late',
 'not what who',
 'yeah yeah jesus',
 'does it hurt',
 'shut up look',
 'ha ha',
 'no way hes mine',
 'not good',
 

In [160]:
X = np.array([w2i_2(i[0]) for i in data])
y = np.array([w2i(i[1]) for i in data])

In [186]:
more_pad=np.ones((y.shape[0],1),dtype=np.int)*2

In [188]:
y_shift=np.concatenate([y[:,1:],more_pad],axis=1)

In [153]:
from keras.layers import LSTM, Input, Dense, Embedding
from keras.models import Sequential, Model

latent_dim = 10
num_tokens = 12

# Encoder
encoder_input = Input(shape=(None,))
encoder_embedding=Embedding(num_tokens, latent_dim)(encoder_input)

encoder=LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c=encoder(encoder_embedding)
encoder_states=[state_h, state_c]

# Decoder
decoder_input=Input(shape=(None,))
decoder_embedding=Embedding(num_tokens, latent_dim)(decoder_input)
decoder_lstm=LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_,_=decoder_lstm(
    decoder_embedding, initial_state=encoder_states)
decoder_dense=Dense(num_tokens, activation="softmax")
decoder_output=decoder_dense(decoder_outputs)

# Model
model=Model([encoder_input, decoder_input], decoder_output)

In [156]:
model.compile(loss="categorical_crossentropy",optimizer="adam")
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, None, 10)     120         input_13[0][0]                   
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, None, 10)     120         input_14[0][0]                   
__________________________________________________________________________________________________
lstm_21 (L